In [ ]:
import json

In [ ]:
%load demo-assume-role-policy-internal.json

In [ ]:
ls

<img src="s3-cross-account.png" class="left"/>

The resources for this lab are an S3 bucket we will refet to as mybucket (actually mybucket-$random-$random
All roles shown above will have the same permissions policy which allows them to access generic S3 resources.

This lab requires an admin_A and admin_B which can create all the necessary resources. For the purposes of instcurtion
we will create resources as needed. If you are running this for strudents, the admin can run setupA.sh and setupB.sh. 

In [ ]:
%%bash --out bash_output
echo $RANDOM


In [ ]:
rand = str(bash_output.strip())
rand

In [ ]:
rand = str(30873)

For this lab, you will need user credentials in two AWS accounts A and B and ~/.aws/credentials with blocks like
```
[profileA_rhs]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx

[profileB_rhs]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx
```

Fill in the values you wish to use for the RHS (right hand side) of profileA and profileB below as in your ~/.aws/credentials file.

In [ ]:
profileA = "pbeta"
profileB = "pdelta"
mybucket1 = "mybucket1-" + rand
mybucket2 = "mybucket2-" + rand
path = "/aws-labs/" # path acts as a prefix to IAM roles and policies. It can be used like tags to list our assets.

In [ ]:
!aws --profile $profileA sts get-caller-identity

In [ ]:
!aws --profile $profileB sts get-caller-identity

Use the output Arn's from above to fill in the values below. arn:aws:iam::...

In [ ]:
principalA_arn = "arn:aws:iam::111111111111:user/kesten"
principalB_arn = "arn:aws:iam::222222222222:user/kesten"
accountA = principalA_arn.split(':')[4]
accountB = principalB_arn.split(':')[4]
accountB

However

In [ ]:
%%bash
aws --profile "${profileA}" sts get-caller-identity
aws --profile "${profileB}" sts get-caller-identity
# The following evaluate $profileA to null
# aws --profile $profileA sts get-caller-identity
# aws --profile $profileB sts get-caller-identity

As you can see from above, we must take care as bash magic in cells may not behave as we expect. The output of the cell above should correspond to profileB, but it does not. The moral is, try not to get fancy with bash in jupyter.


## Admin Setup
Use the Arn fields above to fill in the values below. If you do not have admin
credentials, this must be done by the administrator

The following policy is attached to all roles except role-A2 allowing the associated role S3 access to any S3

In [ ]:
roleA1 = "roleA1-" + rand
roleA2 = "roleA2-" + rand
roleB1 = "roleB1-" + rand
roleB2 = "roleB2-" + rand

In [ ]:
# assume_A_policy.json
assume_A_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AssumeRolePolicyForS3ReaderRoleByExternal",
      "Effect": "Allow",
      "Principal": {"AWS": [principalA_arn]},
      "Action": "sts:AssumeRole"
    }]
}
assume_A_policy

In [ ]:
assume_A_policy_str = json.dumps(assume_A_policy)
assume_A_policy_str

In [ ]:
!aws --profile $profileB sts get-caller-identity

In [ ]:
!echo $profileA + $roleA1 $path


In [ ]:
!aws --profile $profileA iam create-role --role-name $roleA1 \
                                        --path $path \
                                        --assume-role-policy-document assume_A_policy_str

In [ ]:
!aws --profile $profileA iam create-role --role-name $roleA2 \
                                        --path $path 
                                        --assume-role-policy-document assume_A_policy_str

In [ ]:
!aws --profile $profileB iam create-role --role-name $roleB1 \
                                        --path $path \
                                        --assume-role-policy-document assume_B_policy_str

In [ ]:
!aws --profile $profileB iam create-role --role-name $roleB2 \
                                        --path $path \
                                        --assume-role-policy-document assume_B_policy_str

Create the buckets for the lab. Notie that unlike the IAM resources created, the bucket ARN does not include the account ID.

In [ ]:
!aws --profile $profileA s3api create-bucket --bucket $mybucket1
!aws --profile $profileA s3api create-bucket --bucket $mybucket2 \
                --create-bucket-configuration LocationConstraint=us-west-1

In [ ]:
!aws --profile $profileA s3 cp demo-vars.sh s3://$mybucket1/
!aws --profile $profileA s3 cp assume_A_policy.json s3://$mybucket2/

In [ ]:
!aws --profile $profileA s3 ls s3://$mybucket1

In [ ]:
tagset = 'TagSet=[{Key=path,Value=' + path + '},{Key=rand,Value=' + rand + '}]'
tagset

In [ ]:
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket1 \
                --tagging $tagset
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket2 \
                --tagging $tagset

We have written a function awsas which is aws_run_as.sh
in this repo which does the following:<br><br>
awsas --profile profileB roleB1 [aws command args] <br><br>
This does the magic of using user/service credentials in profileB to 
assume roleB1 and then run commands. Without this function, you have to get the response 
for assume-role and put them into environment variables or ~/.aws/credentials profile each time
as described [here](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-role.html). 
This is a very new beta script so drop into a bash shell if anything goes wrong :)<br>
Let's try it out!

In [ ]:
!awsas  --debug --profile $profileB $roleB1  sts get-caller-identity

### Admin setup of managed policy

In [ ]:
def jdump(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, sort_keys=True, indent=4 * ' ')

In [ ]:
# permission-policy.json
iam_permission_policy_for_s3 = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "PermissionPolicyForS3Access",
      "Effect": "Allow",
      "Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1),
                  "arn:aws:s3:::{}/*".format(mybucket2), 
                   "arn:aws:s3:::{}".format(mybucket2)]
    }]
}
jdump(iam_permission_policy_for_s3 , "iam_permission_policy_for_s3.json")

Since we want to attach this policy to multiple roles, we will need to create a managed policy, 
rather than an inline policy which is embeded in the role.

In [ ]:
!aws --profile $profileA iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                  --path "/aws-labs/" \
                                  --policy-document file://iam_permission_policy_for_s3.json

We must create the policy in each AWS account so repeat for B

In [ ]:
!aws --profile $profileB iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                --path "/aws-labs/" \
                                --policy-document file://iam_permission_policy_for_s3.json

## Exercises

## 1. Confirm no access for roles with no permissions and default (no) bucket policy
So far, we have created policies and roles. When we create a role, we must include the assume-role 
trust policy which says who can assume the role. But so far it is just an empty container for us to 
put permissions policies in. Before we attach a bucket policy, let's check that we we can do with the roles
as they are.

In [ ]:
!awsas  --profile $profileA $roleA1 s3 ls $mybucket1

## 2. Check a role with no permissions can access a bucket with the right bucket policy

In [ ]:
# mybucket1_policy.json
mybucket1_policy = {
  "Version": "2012-10-17",
  "Statement": [{
      "Sid": "BucketPolicyForS3MyBucket1",
      "Effect": "Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountA), 
                            "arn:aws:iam::{}:role/{}".format(accountA, roleA2)]},
      "Action": ["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1)]

    }]
}
jdump(mybucket1_policy , "mybucket1_policy.json")
mybucket1_policy_str = json.dumps(mybucket1_policy).replace(' ', '')

Great, now lets attach a permissions policy to roleA1. We still have no roles

In [ ]:
mybucket1_policy_str

In [ ]:
!echo $mybucket1 $mybucket1_policy_str $profileA

The following should work but fails

In [ ]:
!aws --profile pbeta s3api put-bucket-policy --bucket $mybucket1 \
--policy $mybucket1_policy_str

In [ ]:
!aws --profile pbeta s3api put-bucket-policy --bucket $mybucket1 \
--policy '{"Version":"2012-10-17","Statement":[{"Sid":"AssumeRolePolicyForS3ReaderRoleB","Effect":"Allow","Principal":{"AWS":["arn:aws:iam::111111111111:root","arn:aws:iam::111111111111:role/roleA2-30873"]},"Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],"Resource":["arn:aws:s3:::mybucket1-30873/*","arn:aws:s3:::mybucket1-30873"]}]}'

If we eliminate all the variable expansions then the command works fine.

In [ ]:
!aws --profile pbeta s3api put-bucket-policy --bucket mybucket1-30873 --policy '{"Version":"2012-10-17","Statement":[{"Sid":"AssumeRolePolicyForS3ReaderRoleB","Effect":"Allow","Principal":{"AWS":["arn:aws:iam::111111111111:root","arn:aws:iam::111111111111:role/roleA2-30873"]},"Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],"Resource":["arn:aws:s3:::mybucket1-30873/*","arn:aws:s3:::mybucket1-30873"]}]}'

## Switch back to Readme.md for now

In [ ]:
# mybucket2_policy.json
mybucket2_policy = {
  "Version": "2012-10-17",
  "Statement": [{
      "Sid": "BucketPolicyForS3MyBucket2",
      "Effect": "Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountB),
                           "arn:aws:iam::{}:role/{}".format(accountB, roleB2)]},
      "Action": ["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket2), 
                   "arn:aws:s3:::{}".format(mybucket2)]
    }]
}
jdump(mybucket2_policy , "mybucket2_policy.json")